# TD : `keras` & perceptron multi-couches

Romain Tavenard
Creative Commons CC BY-NC-SA

Dans cette séance, nous nous focaliserons sur la création et l'étude de modèles
de type perceptron multi-couches à l'aide de la librairie `keras`.

Pour cela, vous utiliserez la classe de modèles `Sequential()` de `keras`.
Voici ci-dessous un exemple de définition d'un tel modèle :

In [5]:
from keras.models import Sequential
from keras.layers import Dense

#1. définir les couches et les ajouter l'une après l'autre au modèle
premiere_couche = Dense(units=12, activation="relu", input_dim=24)
couche_cachee = Dense(units=12, activation="sigmoid")
couche_sortie = Dense(units=3, activation="linear")

model = Sequential()
model.add(premiere_couche)
model.add(couche_cachee)
model.add(couche_sortie)

Using TensorFlow backend.


2. Spécifier l'algo d'optimisation et la fonction de risque à optimiser

Fonctions de risque classiques :
 * "mse" en régression,
 * "categorical_crossentropy" en classification multi-classes
 * "binary_crossentropy" en classification binaire
 
 On peut y ajouter des métriques supplémentaires (ici taux de bonne
 classifications)

In [6]:
model.compile(optimizer="sgd", loss="mse", metrics=["accuracy"])

3. Lancer l'apprentissage

In [8]:
#model.fit(X_train, y_train, verbose=2, epochs=10, batch_size=200)

# Préparation des données

Pour ce TD, nous vous proposons d'utiliser les fonctions suivantes pour préparer
vos données à l'utilisation dans `keras` :

In [18]:
from sklearn.preprocessing import MinMaxScaler
from keras.datasets import mnist, boston_housing
from keras.utils import to_categorical
import numpy as np

In [19]:
def prepare_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape((x_train.shape[0], -1))
    x_test = x_test.reshape((x_test.shape[0], -1))
    scaler = MinMaxScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return x_train, x_test, y_train, y_test

In [29]:
def prepare_boston():
    (x_train, y_train), (x_test, y_test) = boston_housing.load_data()
    
    scaler_x = MinMaxScaler()
    scaler_x.fit(x_train)
    x_train = scaler_x.transform(x_train)
    x_test = scaler_x.transform(x_test)
    scaler_y = MinMaxScaler()
    scaler_y.fit(y_train[:,np.newaxis])
    y_train = scaler_y.transform(y_train[:,np.newaxis])
    y_test = scaler_y.transform(y_test[:,np.newaxis])
    return x_train, x_test, y_train, y_test

In [30]:
x_train, x_test, y_train, y_test = prepare_boston()
x_train.shape, y_train.shape

((404, 13), (404, 1))

In [31]:

x_train.shape, y_train.shape

11493376/11490434 [==============================] - 8s 1us/step


/opt/miniconda3/envs/ml-2018/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


((60000, 784), (60000, 10))

1. Observez le code des fonctions `prepare_mnist` et `prepare_boston`.
Que font ces fonctions ? Quelles sont les dimensions des matrices / vecteurs à
la sortie ? S'agit-il de problèmes de classification ou de régression ?

# Premiers réseaux de neurone

2. Chargez le jeu de données MNIST et apprenez un premier modèle sans couche
cachée avec une fonction d'activation raisonnable pour les neurones de la couche
de sortie. Pour cette question comme pour les suivantes, limitez vous à un
nombre d'itérations de l'ordre de 10 : ce n'est absolument pas réaliste, mais
cela vous évitera de perdre trop de temps à scruter l'apprentissage de vos
modèles.

In [36]:
# regression logicstique
x_train, x_test, y_train, y_test = prepare_mnist()
x_train.shape, y_train.shape

/opt/miniconda3/envs/ml-2018/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


((60000, 784), (60000, 10))

In [37]:
input_layer = Dense(units=y_train.shape[1], 
                    activation="softmax", 
                    input_dim=x_train.shape[1])

model = Sequential()
model.add(input_layer)
model.compile(optimizer="adam",
             loss="categorical_crossentropy",
             metrics=["accuracy"])

model.fit(x_train, y_train, epochs=10, batch_size=256, verbose=2)

Epoch 1/10
 - 1s - loss: 0.8476 - acc: 0.7997
Epoch 2/10
 - 1s - loss: 0.4256 - acc: 0.8896
Epoch 3/10
 - 1s - loss: 0.3605 - acc: 0.9026
Epoch 4/10
 - 1s - loss: 0.3304 - acc: 0.9096
Epoch 5/10
 - 1s - loss: 0.3125 - acc: 0.9143
Epoch 6/10
 - 1s - loss: 0.3005 - acc: 0.9167
Epoch 7/10
 - 1s - loss: 0.2917 - acc: 0.9190
Epoch 8/10
 - 1s - loss: 0.2851 - acc: 0.9203
Epoch 9/10
 - 1s - loss: 0.2796 - acc: 0.9227
Epoch 10/10
 - 1s - loss: 0.2754 - acc: 0.9232


3. Comparez les performances de ce premier modèle è celle de modèles avec
respectivement 1, 2 et 3 couches cachées de 128 neurones chacune. Vous
utiliserez la fonction ReLU (`"relu"`) comme fonction d'activation pour les
neurones des couches cachées.

In [39]:
input_layer = Dense(units=128, 
                    activation="relu", 
                    input_dim=x_train.shape[1])

second_layer = Dense(units=128, 
                    activation="relu", 
                    input_dim=x_train.shape[1])


third_layer = Dense(units=y_train.shape[1], 
                    activation="softmax")

model = Sequential()
model.add(input_layer)
model.add(second_layer)
model.add(third_layer)

model.compile(optimizer="adam",
             loss="categorical_crossentropy",
             metrics=["accuracy"])

model.fit(x_train, y_train, epochs=10, batch_size=256, verbose=2)

Epoch 1/10
 - 1s - loss: 0.3882 - acc: 0.8927
Epoch 2/10
 - 1s - loss: 0.1531 - acc: 0.9551
Epoch 3/10
 - 1s - loss: 0.1091 - acc: 0.9680
Epoch 4/10
 - 1s - loss: 0.0836 - acc: 0.9750
Epoch 5/10
 - 1s - loss: 0.0672 - acc: 0.9797
Epoch 6/10
 - 1s - loss: 0.0548 - acc: 0.9836
Epoch 7/10
 - 1s - loss: 0.0461 - acc: 0.9857
Epoch 8/10
 - 1s - loss: 0.0377 - acc: 0.9887
Epoch 9/10
 - 1s - loss: 0.0302 - acc: 0.9912
Epoch 10/10
 - 1s - loss: 0.0249 - acc: 0.9927


4. On peut obtenir le nombre de paramètres d'un modèle à l'aide de la
méthode `count_params()`. Comptez ainsi le nombre de paramètres du modèle à
3 couches cachées et définissez un modèle à une seule couche cachée ayant un
nombre comparable de paramètres. Parmi ces deux modèles, lequel semble le plus
performant ?

In [41]:
input_layer = Dense(units=128, 
                    activation="relu", 
                    input_dim=x_train.shape[1])

second_layer = Dense(units=128, 
                    activation="relu", 
                    input_dim=x_train.shape[1])


third_layer = Dense(units=y_train.shape[1], 
                    activation="softmax")

model = Sequential()
model.add(input_layer)
model.add(second_layer)
model.add(third_layer)

model.compile(optimizer="adam",
             loss="categorical_crossentropy",
             metrics=["accuracy"])

model.fit(x_train, y_train, 
          epochs=10, batch_size=256, 
          verbose=2, validation_split=0.1)

y_pred = model.predict(x_train)

confusion_matrix(y_train.argmax(axis=1), y_pred.argmax(axis=1))

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
 - 1s - loss: 0.4320 - acc: 0.8777 - val_loss: 0.1668 - val_acc: 0.9538
Epoch 2/10
 - 1s - loss: 0.1710 - acc: 0.9500 - val_loss: 0.1158 - val_acc: 0.9682
Epoch 3/10
 - 1s - loss: 0.1191 - acc: 0.9654 - val_loss: 0.1007 - val_acc: 0.9722
Epoch 4/10
 - 1s - loss: 0.0897 - acc: 0.9737 - val_loss: 0.0898 - val_acc: 0.9718
Epoch 5/10
 - 1s - loss: 0.0723 - acc: 0.9788 - val_loss: 0.0835 - val_acc: 0.9755
Epoch 6/10
 - 1s - loss: 0.0599 - acc: 0.9823 - val_loss: 0.0780 - val_acc: 0.9782
Epoch 7/10
 - 1s - loss: 0.0484 - acc: 0.9855 - val_loss: 0.0763 - val_acc: 0.9783
Epoch 8/10
 - 1s - loss: 0.0390 - acc: 0.9890 - val_loss: 0.0844 - val_acc: 0.9770
Epoch 9/10
 - 1s - loss: 0.0326 - acc: 0.9902 - val_loss: 0.0761 - val_acc: 0.9762
Epoch 10/10
 - 1s - loss: 0.0280 - acc: 0.9920 - val_loss: 0.0738 - val_acc: 0.9798


In [43]:
model.count_params()

118282

# Utilisation d'un jeu de validation

Bien entendu, les observations faites plus haut ne sont pas suffisantes,
notamment parce qu'elles ne permettent pas de se rendre compte de l'ampleur du
phénomène de sur-apprentissage.

Pour y remédier, `keras` permet de fixer, lors de l'appel à la méthode `fit()`,
une fraction du jeu d'apprentissage à utiliser pour la validation.
Jetez un oeil
[ici](https://keras.io/getting-started/faq/#how-is-the-validation-split-computed)
pour comprendre comment les exemples de validation sont
choisis.

5. Répétez les comparaisons de modèles ci-dessus en vous focalisant sur le taux
de bonnes classifications obtenu sur le jeu de validation (vous prendrez 30\%
    du jeu d'apprentissage pour votre validation).

# Régularisation et _Drop-Out_

6. Appliquez une régularisation de type $L_1$ à chacune des couches de votre
réseau. L'aide disponible [ici](https://keras.io/regularizers/) devrait
vous aider.

7. Au lieu de la régularisation $L_1$, choisissez de mettre en place une
stratégie de [_Drop-Out_](https://keras.io/layers/core/#dropout) pour aider à la
régularisation de votre réseau.
Vous éteindrez à chaque étape 10\% des poids de votre réseau.

# Algorithme d'optimisation et vitesse de convergence

8. Modifiez la méthode d'optimisation choisie. Vous pourrez notamment essayer
les algorithmes `"rmsprop"` et `"adam"`, reconnus pour leurs performances.

9. En utilisant l'aide fournie [ici](https://keras.io/optimizers/), faites
varier le paramètre `lr` (_learning rate_) à l'extrême pour observer :

* l'instabilité des performances lorsque celui-ci est trop grand ;
* la lenteur de la convergence lorsque celui-ci est trop petit.

# Modèles `keras` dans `sklearn`

Il est possible de transformer vos modèles `keras` (en tout cas, ceux qui sont
    de type `Sequential`) en modèles `sklearn`. Cela a notamment pour avantage
de vous permettre d'utiliser les fonctionnalités de sélection de modèles vues
lors du TD précédent.

Pour cela, vous devrez utiliser au choix l'une des classes `KerasClassifier` ou
`KerasRegressor` (selon le problème de _machine learning_ auquel vous êtes
    confronté) du module `keras.wrappers.scikit-learn`.

Le principe de fonctionnement de ces deux classes est le même :

In [ ]:
clf = KerasClassifier(build_fn=ma_fonction, param1=12, param2="sgd", ...)
clf.fit(X, y)
clf.predict(X_test)

Une fois construit, l'objet `clf` s'utilise donc exactement comme un classifieur
`sklearn`.
L'attribut `build_fn` prend le nom d'une fonction qui retourne un modèle
`keras`. Les autres paramètres passés lors de la construction du classifieur
peuvent être :

* des paramètres de votre fonction `ma_fonction` ;
* des paramètres passés au modèle lors de son apprentissage (appel à la
    méthode `fit()`).

10. Créez un réseau à deux couches cachées transformé en objet `sklearn` en
spécifiant, lors de sa construction, le nombre d'itérations et la taille des
_batchs_ de votre descente de gradient par _mini-batchs_. Vous pourrez
utiliser la méthode `score()` des objets `sklearn` pour évaluer ce modèle.

11. Utilisez les outils de validation croisée de `sklearn` pour choisir entre
les algorithmes d'optimisation `"rmsprop"` et `"sgd"`.

# La notion de `Callback`

Les _Callbacks_ sont des outils qui, dans `keras`, permettent d'avoir un oeil
sur ce qui se passe lors de l'apprentissage et, éventuellement, d'agir sur cet
apprentissage.

Le premier _callback_ auquel vous pouvez accéder simplement est retourné
lors de l'appel à la méthode `fit()` (sur un objet `keras`, pas `sklearn`). Ce
_callback_ est un objet qui possède un attribut `history`. Cet attribut est un
dictionnaire dont les clés sont les métriques suivies lors de l'apprentissage.
À chacune de ces clés est associé un vecteur indiquant comment la quantité en
question a évolué au fil des itérations.

12. Tracez les courbes d'évolution du taux de bonnes classifications sur les
jeux d'entrainement et de validation.

La mise en place d'autres _callbacks_ doit être explicite. Elle se fait en
passant une liste de _callbacks_ lors de l'appel à la méthode `fit()`.
Lorsque l'apprentissage prend beaucoup de temps, la méthode précédente n'est pas
satisfaisante car il est nécessaire d'attendre la fin du processus
d'apprentissage avant de visualiser ces courbes. Dans ce cas, le _callback_
[`TensorBoard`](https://keras.io/callbacks/#tensorboard) peut s'avérer très
pratique.

13. Visualisez dans une page TensorBoard l'évolution des métriques `"loss"`
et `"accuracy"` lors de l'apprentissage d'un modèle.

De même, lorsque l'apprentissage est long, il peut s'avérer souhaitable
d'enregistrer des modèles intermédiaires, dans le cas où un plantage arriverait
par exemple. Cela peut se faire à l'aide du _callback_
[`ModelCheckpoint`](https://keras.io/callbacks/#modelcheckpoint).

14. Mettez en place un enregistrement des modèles intermédiaires toutes les 2
itérations, en n'enregistrant un modèle que si le risque calculé sur le jeu de
validation est plus faible que celui de tous les autres modèles enregistrés
aux itérations précédentes.

15. Mettez en oeuvre une politique d'arrêt précoce de l'apprentissage au cas où
le risque calculé sur le jeu de validation n'a pas diminué depuis au moins 5
itérations (en utilisant le _callback_
[`EarlyStopping`](https://keras.io/callbacks/#earlystopping)).

# Exercice de synthèse

16. Mettez en place une validation croisée pour choisir la structure (nombre de
    couches, nombre de neurones par couche) et l'algorithme d'optimisation
    idoines pour le problème lié au jeu de données _Boston Housing_ (pour lequel
        une fonction de préparation des données est fournie dans le module
        `dataset_utils`).